In [43]:
import numpy as np
import pandas as pd
import random
import math as mt

In [44]:
df = pd.read_csv('output.csv')

In [45]:
df

,Unnamed: 0,Name,Suit,Points
0,0,Queen of Hearts,Hearts,0.03
1,1,9 of Diamonds,Diamonds,2.00
2,2,10 of Spades,Spades,1.00
3,3,Queen of Clubs,Clubs,0.03
4,4,7 of Clubs,Clubs,0.01
5,5,Ace of Hearts,Hearts,1.10
6,6,10 of Diamonds,Diamonds,1.00
7,7,9 of Clubs,Clubs,2.00
8,8,10 of Clubs,Clubs,1.00
9,9,8 of Hearts,Hearts,0.02


In [46]:
df = df.drop('Unnamed: 0', axis = 1)

In [47]:
class Player():
    def __init__(self, cards, name):
        self.cards = cards
        self.colour_made = False
        self.points_made = 0
        self.name = name
        self.hands_won = []
        self.last_hand_won = False
        
class Robot():
    def __init__(self, cards, name):
        self.cards = cards
        self.colour_made = False
        self.points_made = 0
        self.name = name
        self.hands_won = []
        self.last_hand_won = False
      
class Game():
    def __init__(self):
        self.color = None
        self.last_hand_winner = None
        self.digged = False


In [48]:
class Bid():
    def __init__(self, b1, b2):
        self.how_much_b1 = 17
        self.how_much_b2 = 17
        self.b1_bid = input(f"Do you want to bid {b1.name}? Y/N: ").lower()
        
        if self.b1_bid == "n":
            print(f"{b2.name} wins the bid as {b1.name} passes at {self.how_much_b2}")
            self.winner = b2
        elif self.b1_bid == "y":
            self.b2_bid = input(f"Do you want to bid {b2.name}? Y/N: ").lower()

            if self.b2_bid == "n":
                print(f"{b1.name} wins the bid as {b2.name} passes at {self.how_much_b1}")
                self.winner = b1
            else:
                while self.b1_bid == "y" and self.b2_bid == "y":
                    self.how_much_b1 = int(input(f"How much would you like to bid {b1.name}? (minimum bid is 17): "))
                    self.how_much_b2 = int(input(f"How much would you like to bid {b2.name}? (minimum bid is 17): "))

                    # Ensure minimum bid
                    while self.how_much_b1 < 17 or self.how_much_b2 < 17:
                        print("Invalid bid! The minimum bid is 17.")
                        self.how_much_b1 = int(input(f"How much would you like to bid {b1.name}? (minimum bid is 17): "))
                        self.how_much_b2 = int(input(f"How much would you like to bid {b2.name}? (minimum bid is 17): "))

                    # Determine winner of the bid
                    if self.how_much_b1 < self.how_much_b2:
                        self.b1_bid = input(f"{b2.name} is winning the bid. Do you want to continue, {b1.name}? Y/N: ").lower()
                        if self.b1_bid == "n":
                            print(f"{b2.name} wins the bid as {b1.name} passes at {self.how_much_b2}")
                            self.winner = b2
                    elif self.how_much_b1 > self.how_much_b2:
                        self.b2_bid = input(f"{b1.name} is winning the bid. Do you want to continue, {b2.name}? Y/N: ").lower()
                        if self.b2_bid == "n":
                            print(f"{b1.name} wins the bid as {b2.name} passes at {self.how_much_b1}")
                            self.winner = b1


In [49]:
game = Game()

In [50]:
def shuffle_deck():
    global df
    df = df.sample(frac = 1, ignore_index = True)

In [51]:
def distribute_cards():
    global df
    global player, robot_1, robot_2, robot_3
    player = Player(df[0:8], input("What is your name? "))
    robot_1 = Robot(df[8:16], 'Robot 1')
    robot_2 = Robot(df[16:24], 'Robot 2')
    robot_3 = Robot(df[24:32], 'Robot 3')

In [52]:
def play_card(name, player):
    global current_hand
    current_hand = pd.concat([current_hand, df[df['Name'] == name]], ignore_index = True)
    # pd.concat([new_data, data], ignore_index=True)
    player.cards = player.cards.drop(player.cards[player.cards['Name'] == name].index)

In [80]:
def setup():
    
    shuffle_deck() # Shuffling all the cards
    distribute_cards() # Distributing all the cards
    print("Nice to meet you {}".format(player.name))
    print("Welcome to our newly made game of 29. We hope you enjoy it")
    print(player.cards[:4]) # Showing the player their first 4 cards
    global game,player_bid,r1_bid,r2_bid,r3_bid,max_bid
    game = Game()
        
    # BIDDING
    player_bid = int(input('How much do you want to bid?: '))
    
    # robot_1
    r1_bid = 0
    if len(robot_1.cards[:4]['Suit'].mode() == 1):
        r1_mode_suit = robot_1.cards[:4]['Suit'].mode().iloc[0]
        r1_color = robot_1.cards[:4]['Suit'].value_counts()[r1_mode_suit]
        r1_points = robot_1.cards[:4][robot_1.cards[:4]['Suit'] == r1_mode_suit]['Points'].sum()
        
        if mt.floor(r1_points) == 3 :
            if r1_color == 2: r1_bid = 18
            if r1_color == 3: r1_bid = 19
        if mt.floor(r1_points) == 4 :
            if r1_color == 2: r1_bid = 18
            if r1_color == 3: r1_bid = 19
        if mt.floor(r1_points) == 5:
            if r1_color == 2: r1_bid = 19
            if r1_color == 3: r1_bid = 20
        if mt.floor(r1_points) == 6 :
            if r1_color == 3: r1_bid = 22
        if mt.floor(r1_points) == 6 or mt.floor(r1_points) == 7:
            if r1_color == 4: r1_bid = 22
    
    # robot_2
    r2_bid = 0
    if len(robot_2.cards[:4]['Suit'].mode() == 2):
        r2_mode_suit = robot_2.cards[:4]['Suit'].mode().iloc[0]
        r2_color = robot_2.cards[:4]['Suit'].value_counts()[r2_mode_suit]
        r2_points = robot_2.cards[:4][robot_2.cards[:4]['Suit'] == r2_mode_suit]['Points'].sum()
        
        if mt.floor(r2_points) == 3 :
            if r2_color == 2: r2_bid = 18
            if r2_color == 3: r2_bid = 19
        if mt.floor(r2_points) == 4 :
            if r2_color == 2: r2_bid = 18
            if r2_color == 3: r2_bid = 19
        if mt.floor(r2_points) == 5:
            if r2_color == 2: r2_bid = 19
            if r2_color == 3: r2_bid = 20
        if mt.floor(r2_points) == 6 :
            if r2_color == 3: r2_bid = 22
        if mt.floor(r2_points) == 6 or mt.floor(r2_points) == 7:
            if r2_color == 4: r2_bid = 22
    
    # robot_3
    r3_bid = 0
    if len(robot_3.cards[:4]['Suit'].mode() == 3):
        r3_mode_suit = robot_3.cards[:4]['Suit'].mode().iloc[0]
        r3_color = robot_3.cards[:4]['Suit'].value_counts()[r3_mode_suit]
        r3_points = robot_3.cards[:4][robot_3.cards[:4]['Suit'] == r3_mode_suit]['Points'].sum()
        
        if mt.floor(r3_points) == 3 :
            if r3_color == 2: r3_bid = 18
            if r3_color == 3: r3_bid = 19
        if mt.floor(r3_points) == 4 :
            if r3_color == 2: r3_bid = 18
            if r3_color == 3: r3_bid = 19
        if mt.floor(r3_points) == 5:
            if r3_color == 2: r3_bid = 19
            if r3_color == 3: r3_bid = 20
        if mt.floor(r3_points) == 6 :
            if r3_color == 3: r3_bid = 22
        if mt.floor(r3_points) == 6 or mt.floor(r3_points) == 7:
            if r3_color == 4: r3_bid = 22
    bidding = [player_bid, r1_bid, r2_bid, r3_bid]
    max_bid = max(bidding)
    max_bid_index = bidding.index(max_bid)
    if max_bid <17:
        r3_bid = 17
        winner = robot_3.name
        max_bid = r3_bid
    else:
        if max_bid_index == 0: 
            winner = player.name
        if max_bid_index == 1: 
            winner = robot_1.name
        if max_bid_index == 2: 
            winner = robot_2.name
        if max_bid_index == 3: 
            winner = robot_3.name
    print('{} won the bet and will make the color on {} points.'.format(winner, max_bid))
    if winner == player.name:
        player.colour_made = True
        game.color = input(f"What is the colour {player.name}?")
    if winner == robot_1.name:
        game.color = r1_mode_suit
    if winner == robot_2.name:
        game.color = r2_mode_suit
    if winner == robot_3.name:
        game.color = r3_mode_suit
    game.color = game.color.capitalize()

In [54]:
def flow_chart(r):
   
    global current_hand, game
    global df

    r.suits = list(r.cards["Suit"])
    if len(current_hand) == 0:
        if len(r.cards[r.cards['Points'] == 3]) > 0:
            card_to_play = r.cards[r.cards['Points'] == 3].sample(n = 1, random_state = None)['Name'].iloc[0]
            print('{} plays {}'.format(r.name, card_to_play))
            play_card(card_to_play, r)
        else: 
            card_to_play = r.cards[r.cards['Points'] == r.cards['Points'].min()].sample(n = 1, random_state = None)['Name'].iloc[0]
            print('{} plays {}'.format(r.name, card_to_play))
            play_card(card_to_play,r)
    else: 
        current_suit = current_hand.iloc[0]['Suit']
        
        if current_suit in r.suits:
            how_many = r.suits.count(current_suit)

            if how_many == 1:
                card_to_play = r.cards[r.cards['Suit'] == current_suit]['Name']
                card_to_play = card_to_play.iloc[0]
                print('{} plays {}'.format(r.name, card_to_play))
                play_card(card_to_play, r)

            if how_many > 1:

                if game.digged and game.color in list(current_hand['Suit']):
                    find_in = current_hand[current_hand['Suit'] == game.color]
                    winner_index = find_in[find_in['Points'] == find_in['Points'].max()].index
                elif not game.color in list(current_hand['Suit']) or not game.digged:
                    c_s = current_hand['Suit'].iloc[0]
                    find_in = current_hand[current_hand['Suit'] == c_s]
                    winner_index = find_in[find_in['Points'] == find_in['Points'].max()].index
                    
                if len(r.cards[(r.cards['Suit'] == current_suit) & (r.cards['Points'] == 3)]) == 0:   
                    if not game.digged:    
                        #CHECK WHO IS WINNING
                        if winner_index == len(current_hand) - 1 or winner_index == len(current_hand) - 3:
                            cs = r.cards[r.cards['Suit'] == current_suit]
                            card_to_play = cs[cs['Points'] == cs['Points'].min()].sample(n = 1)['Name']
                            card_to_play = card_to_play.iloc[0]
                            print('{} plays {}'.format(r.name, card_to_play))
                            play_card(card_to_play, r)
                        if winner_index == len(current_hand) - 2:
                            cs = r.cards[r.cards['Suit'] == current_suit]
                            card_to_play = cs[cs['Points'] == cs['Points'].max()].sample(n = 1)['Name']
                            card_to_play = card_to_play.iloc[0]
                            print('{} plays {}'.format(r.name, card_to_play))
                            play_card(card_to_play, r)

                else:
                    card_to_play = r.cards[(r.cards['Suit'] == current_suit) & (r.cards['Points'] == 3)]['Name']
                    card_to_play = card_to_play.iloc[0]
                    print('{} plays {}'.format(r.name, card_to_play))
                    play_card(card_to_play, r)

        if not game.digged and current_suit not in r.suits:
            print(f'{r.name} has revealed the trump card. It is {game.color}')
            if len(r.cards[(r.cards['Suit'] == game.color)]) > 0:
                cs = r.cards[r.cards['Suit'] == game.color]
                card_to_play = cs[cs['Points'] == cs['Points'].min()].sample(n = 1)['Name']
                card_to_play = card_to_play.iloc[0]
                print('{} plays {}'.format(r.name, card_to_play))
                play_card(card_to_play, r)
            else: 
                card_to_play = r.cards[r.cards['Points'] == r.cards['Points'].min()].sample(n = 1, random_state = None)['Name'].iloc[0]
                print('{} plays {}'.format(r.name, card_to_play))
                play_card(card_to_play, r)
            game.digged = True
        elif game.digged and current_suit not in r.suits:
            if len(r.cards[(r.cards['Suit'] == game.color)]) > 0:
                cs = r.cards[r.cards['Suit'] == game.color]
                card_to_play = cs[cs['Points'] == cs['Points'].min()].sample(n = 1)['Name']
                card_to_play = card_to_play.iloc[0]
                print('{} plays {}'.format(r.name, card_to_play))
                play_card(card_to_play, r)
            else: 
                card_to_play = r.cards[r.cards['Points'] == r.cards['Points'].min()].sample(n = 1, random_state = None)['Name'].iloc[0]
                print('{} plays {}'.format(r.name, card_to_play))
                play_card(card_to_play, r)

In [55]:
current_hand = pd.DataFrame(data = {}, columns = df.columns)

In [160]:
def play_game():
       #SETUP THE GAME
    setup()
    import warnings

    warnings.filterwarnings(
        "ignore",
        category=FutureWarning, 
        message="The behavior of DataFrame concatenation with empty or all-NA entries is deprecated"
    )

 
    #DO THE GLOBALS
    global player, robot_1, robot_2, robot_3, game, current_hand, last_hand, winner_index, r1_bid,r2_bid,r3_bid,player_bid, max_bid
    #print(player.cards)
    #PLAYE ALL THE 8 HANDS
    for i in range(0, 8):
        current_hand = pd.DataFrame(data = {}, columns = df.columns)
       
       
        if player.last_hand_won: 
            print(f"Here are your cards {player.name}: \n {player.cards}")
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
            
            play_card(card, player)
            flow_chart(robot_1)
            flow_chart(robot_2)
            flow_chart(robot_3)
            index_selctor = {"player":0,"robot_1":1,"robot_2":2,"robot_3":3}
            print("player")
        elif robot_1.last_hand_won: 
            
            flow_chart(robot_1)
  
            flow_chart(robot_2)
            flow_chart(robot_3)
            print() 
            if game.digged == False and current_hand.iloc[0]["Suit"] not in list(player.cards["Suit"]):
                wanna_dig = input("Do you want to reveal the trump card (y/n)")
                if wanna_dig.upper() =="Y":
                    game.digged = True
                    print(f"The trump card is revealed by the {player.name} and the color is {game.color}")
                
            print(f"Here are your cards {player.name}: \n {player.cards}")
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
            
            
            play_card(card, player)
            index_selctor = {"robot_1":0,"robot_2":1,"robot_3":2,"player":3}
            print("r1")
        elif robot_2.last_hand_won:
          
            flow_chart(robot_2)

            flow_chart(robot_3)
            print() 
            if game.digged == False and current_hand.iloc[0]["Suit"] not in list(player.cards["Suit"]):
                wanna_dig = input("Do you want to reveal the trump card (y/n)")
                if wanna_dig.upper() =="Y":
                    game.digged = True
                    print(f"The trump card is revealed by the {player.name} and the color is {game.color}")
            print(f"Here are your cards {player.name}: \n {player.cards}")
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
          
            play_card(card, player)
            flow_chart(robot_1)
            index_selctor = {"robot_2":0,"robot_3":1,"player":2,"robot_1":3}
            
            print("r2")
        elif robot_3.last_hand_won:
            
            flow_chart(robot_3)
      
            print() 
            if game.digged == False and current_hand.iloc[0]["Suit"] not in list(player.cards["Suit"]):
                wanna_dig = input("Do you want to reveal the trump card (y/n)")
                if wanna_dig.upper() =="Y":
                    game.digged = True
                    print(f"The trump card is revealed by the {player.name} and the color is {game.color}")
            print(f"Here are your cards {player.name}: \n {player.cards}")
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
            
            play_card(card, player)
            flow_chart(robot_1)
            flow_chart(robot_2)
            index_selctor = {"robot_3":0,"player":1,"robot_1":2,"robot_2":3}
            print("r3")
        elif not robot_1.last_hand_won and not robot_2.last_hand_won and not robot_3.last_hand_won and not player.last_hand_won : 
            print(f"Here are your cards {player.name}: \n {player.cards}")
            card = input(f"What card do you wanna play {player.name}")
            card =  card.split(" ")[0].capitalize()+   " of " + card.split(" ")[-1].capitalize()
           
            play_card(card, player)
            flow_chart(robot_1)
            flow_chart(robot_2)
            flow_chart(robot_3)
            index_selctor = {"player":0,"robot_1":1,"robot_2":2,"robot_3":3}
            print("None")
        if game.digged and game.color in list(current_hand['Suit']):
            find_in = current_hand[current_hand['Suit'] == game.color]
            winner_index = find_in[find_in['Points'] == find_in['Points'].max()].index
        elif not game.color in list(current_hand['Suit']) or not game.digged:
            c_s = current_hand['Suit'].iloc[0]
            find_in = current_hand[current_hand['Suit'] == c_s]
            winner_index = find_in[find_in['Points'] == find_in['Points'].max()].index
        if winner_index == index_selctor["player"]: 
                for c in list(current_hand["Name"]): 
                    player.hands_won.append(c)
                player.last_hand_won = True
                robot_1.last_hand_won = False
                robot_2.last_hand_won = False
                robot_3.last_hand_won = False
                current_hand = pd.DataFrame(data = {}, columns = df.columns)
        elif winner_index == index_selctor["robot_1"]: 
                for c in  list(current_hand["Name"]):
                    robot_1.hands_won.append(c)
                player.last_hand_won = False
                robot_1.last_hand_won = True
                robot_2.last_hand_won = False
                robot_3.last_hand_won = False
                current_hand = pd.DataFrame(data = {}, columns = df.columns)
        elif winner_index == index_selctor["robot_2"]: 
                for c in list(current_hand["Name"]):
                    robot_2.hands_won.append(c)
                player.last_hand_won = False
                robot_1.last_hand_won = False
                robot_2.last_hand_won = True
                robot_3.last_hand_won = False
                current_hand = pd.DataFrame(data = {}, columns = df.columns)
        elif winner_index == index_selctor["robot_3"]: 
                for c in list(current_hand["Name"]): 
                    robot_3.hands_won.append(c)
                player.last_hand_won = False
                robot_1.last_hand_won = False
                robot_2.last_hand_won = False
                robot_3.last_hand_won = True
                current_hand = pd.DataFrame(data = {}, columns = df.columns)
        
        
        
        last_hand =  pd.DataFrame(data = {}, columns = df.columns)
        last_hand = current_hand
        current_hand = pd.DataFrame(data = {}, columns = df.columns)
    

    import math as mt

    sprites = [player,robot_1,robot_2,robot_3]
    for sprite in sprites:
        sprite.points_made = 0
    # Calculate points for each sprite
    for sprite in sprites:
        for s_c in sprite.hands_won:
            points = df[df["Name"] == s_c]["Points"] 
            sprite.points_made += points.iloc[0]  # Extract the first value
        if sprite.last_hand_won:
            sprite.points_made += 1

        round(sprite.points_made)

    # Calculate team points
    team_1_points = player.points_made + robot_2.points_made
    team_2_points =robot_1.points_made + robot_3.points_made

    # Determine the winning team
    bids = {
        player_bid: team_1_points,
        r1_bid: team_2_points,
        r2_bid: team_1_points,
        r3_bid: team_2_points,
    }

    if max_bid in bids:
        if bids[max_bid] >= max_bid:
            print(f"Team_{1 if max_bid in [player_bid, r2_bid] else 2} wins!")
        else:
            print(f"Team_{1 if max_bid in [player_bid, r2_bid] else 2} loses!")



In [88]:
play_game()

What is your name? bumbo 
Nice to meet you bumbo 
Welcome to our newly made game of 29. We hope you enjoy it
              Name      Suit  Points
0  Ace of Diamonds  Diamonds    1.10
1      7 of Hearts    Hearts    0.01
2       8 of Clubs     Clubs    0.02
3   Queen of Clubs     Clubs    0.03
How much do you want to bid?: 18
Robot 3 won the bet and will make the color on 19 points.
Here are your cards bumbo : 
               Name      Suit  Points
0  Ace of Diamonds  Diamonds    1.10
1      7 of Hearts    Hearts    0.01
2       8 of Clubs     Clubs    0.02
3   Queen of Clubs     Clubs    0.03
4     Ace of Clubs     Clubs    1.10
5     10 of Spades    Spades    1.00
6       7 of Clubs     Clubs    0.01
7      9 of Spades    Spades    2.00
What card do you wanna play bumbo 7 of Hearts
Robot 1 plays Jack of Hearts
Robot 2 plays Queen of Hearts
Robot 3 plays 8 of Hearts
None
Robot 1 plays 8 of Spades
Robot 2 plays 7 of Spades
Robot 3 plays Jack of Spades

Here are your cards bumbo : 
     

In [78]:
    sprites = [player,robot_1,robot_2,robot_3]
    for sprite in sprites:
        for s_c in sprite.cards:
            sprite.points_made += df[df["Name"] == s_c]["Points"]
        if sprite.last_hand_won:
            sprite.points_made+=1
    
    team_1_points=mt.floor(player.points_made + robot_2.points_made)
    team_2_points = mt.floor(robot_1.points_made + robot_3.points_made)
    
    if max_bid == r1_bid:
        if (team_2_points >= r1_bid).bool:
            print("Team_2 wins!")
        else:
            print("team_2 loses!")
    if max_bid == r2_bid:
        if (team_1_points >= r2_bid).bool:
            print("Team_1 wins!")
        else:
            print("team_1 loses!")
    if max_bid == r3_bid:
        if (team_2_points >= r3_bid).bool:
            print("Team_2 wins!")
        else:
            print("team_2 loses!")
    if max_bid == player_bid:
        if (team_1_points >= player_bid).bool:
            print("Team_1 wins!")
        else:
            print("team_1 loses!")


TypeError: cannot convert the series to <class 'float'>

In [60]:
mt.floor(player.points_made + robot_2.points_made)

TypeError: cannot convert the series to <class 'float'>

Series([], Name: Points, dtype: float64)

In [62]:
player.points_made

Series([], Name: Points, dtype: float64)

In [64]:
robot_3.points_made

Series([], Name: Points, dtype: float64)

In [67]:
for sprite in sprites:
    print(sprite.hands_won)

[]
['Jack of Diamonds', '8 of Hearts', 'Queen of Diamonds', '7 of Diamonds', 'Jack of Hearts', '9 of Hearts', '7 of Hearts', 'Ace of Hearts', '10 of Diamonds', 'King of Diamonds', '8 of Diamonds', '10 of Hearts']
['8 of Clubs', '7 of Clubs', '10 of Clubs', 'Jack of Clubs']
['7 of Spades', 'Ace of Spades', 'Jack of Spades', '10 of Spades', '8 of Spades', 'Ace of Clubs', 'King of Spades', 'King of Clubs', 'Queen of Hearts', 'Queen of Clubs', 'Queen of Spades', 'Ace of Diamonds', 'King of Hearts', '9 of Clubs', '9 of Spades', '9 of Diamonds']


In [76]:
df[df["Name"] == "Jack of Hearts"]['Points']

11    3.0
Name: Points, dtype: float64

In [159]:
import math as mt

sprites = [player,robot_1,robot_2,robot_3]
for sprite in sprites:
    sprite.points_made = 0
# Calculate points for each sprite
for sprite in sprites:
    for s_c in sprite.hands_won:
        points = df[df["Name"] == s_c]["Points"] 
        sprite.points_made += points.iloc[0]  # Extract the first value
    if sprite.last_hand_won:
        sprite.points_made += 1
        
    round(sprite.points_made)

# Calculate team points
team_1_points = player.points_made + robot_2.points_made
team_2_points =robot_1.points_made + robot_3.points_made

# Determine the winning team
bids = {
    player_bid: team_1_points,
    r1_bid: team_2_points,
    r2_bid: team_1_points,
    r3_bid: team_2_points,
}

if max_bid in bids:
    if bids[max_bid] >= max_bid:
        print(f"Team_{1 if max_bid in [player_bid, r2_bid] else 2} wins!")
    else:
        print(f"Team_{1 if max_bid in [player_bid, r2_bid] else 2} loses!")


Team_2 loses!


In [158]:
team_1_points

12

In [153]:
robot_3.points_made

14.329999999999995

In [154]:
robot_2.points_made

2.06

In [155]:
df[df["Name"] == "Jack of Hearts"]["Points"]

8    3.0
Name: Points, dtype: float64

In [113]:
for sprite in sprites:
    for s_c in sprite.cards:
        points = df[df["Name"] == s_c]["Points"]

In [114]:
points

Series([], Name: Points, dtype: float64)

In [115]:
points = df[df["Name"] == s_c]["Points"]

In [116]:
df[df["Name"] == s_c]

,Name,Suit,Points


In [117]:
for sprite in sprites:
    print(sprite.cards)

Empty DataFrame
Columns: [Name, Suit, Points]
Index: []
Empty DataFrame
Columns: [Name, Suit, Points]
Index: []
Empty DataFrame
Columns: [Name, Suit, Points]
Index: []
Empty DataFrame
Columns: [Name, Suit, Points]
Index: []


In [130]:
mt.floor

SyntaxError: invalid syntax (1096276865.py, line 1)